<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_1_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción
gpu----------

Por lo general, las computadoras se utilizan para compilar y analizar los resultados de encuestas y estudios de opinión.

El siguiente cuaderno calcula la **moda** de los **N** valores de los elementos de un vector (vector_resultados), lo hace en forma secuencial y utilizando el procesador CPU. Cada elemento del vector se inicializa con resultados aleatorios de puntajes (numeros enteros entre 0 y 9).

El algoritmo se basa en un ejemplo práctico del libro "C/C++ Cómo Programar"[3] 

Su objetivo es aprender a utilizar Python[2] en la plataforma Colab [1] y la programación secuencial.

#2. Armado de ambiente

Instalación de modulo CUDA  de Python.

In [2]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 14.5MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=f1a917f7870783f78da320765cb5d85485bc9408ed9eb6b87f3563a55b63af8e
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=8be74dc80689b163d209b344887cb41b2864ac06b5a4b671dbcc8c73f1766af9
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo


In [8]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_elementos =   500000#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from datetime import datetime
from   pycuda.compiler import SourceModule

tiempo_total = datetime.now()

# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) \
              * 1000 + dt.microseconds / 1000.0
try:
    # CPU: Inicializo el vector resultados con puntajes (de 0 a 9) aleatorios
    # con cantidad de elementos ingresada.
    vector_resultados_cpu = np.random.randint(0, 10, size = cantidad_elementos)
    vector_resultados_cpu = np.array(vector_resultados_cpu, dtype=np.int32)
    
    # vector_resultados_cpu.astype(np.int32())

    # Inicializo en cero el vector frecuencia (puntajes de 0 a 9)
    vector_frecuencias_cpu = [0 for i in range(10)]
    vector_frecuencias_cpu = np.array(vector_frecuencias_cpu, dtype=np.int32)
    # vector_frecuencias_cpu.astype( numpy.int32() )
    print("-------------------------------------------------")
    print("vector respuestas cpu:")
    print(vector_resultados_cpu)
    

    # CPU - reservo la memoria GPU.
    vector_resultados_gpu = cuda.mem_alloc(vector_resultados_cpu.nbytes)
    vector_frecuencias_gpu = cuda.mem_alloc(vector_frecuencias_cpu.nbytes)

    # GPU - Copio la memoria al GPU.
    cuda.memcpy_htod(vector_resultados_gpu, vector_resultados_cpu)
    cuda.memcpy_htod(vector_frecuencias_gpu, vector_frecuencias_cpu)

    # CPU - Defino la función kernel que ejecutará en GPU.
    module = SourceModule("""
    __global__ void kernel_moda(int n, int *frecuencias , int *resultados)
    {
        int idx = threadIdx.x + blockIdx.x*blockDim.x;

        if(idx < n)
        {
            ++frecuencias[resultados[idx]];        
        }
    }
    """)

    # CPU - Genero la función kernel.
    kernel = module.get_function("kernel_moda")
    
    tiempo_gpu = datetime.now()
    
    dim_hilo = 256
    dim_bloque = np.int( (cantidad_elementos + dim_hilo - 1) / dim_hilo )
    print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
    
    # GPU - Ejecuta el kernel.
    kernel( np.int32(cantidad_elementos), \
            vector_frecuencias_gpu, \
            vector_resultados_gpu, \
            block=( dim_hilo, 1, 1 ), grid=(dim_bloque, 1,1) )

    tiempo_gpu = datetime.now() - tiempo_gpu

    # GPU - Copio el resultado desde la memoria GPU.
    cuda.memcpy_dtoh( vector_frecuencias_cpu, vector_frecuencias_gpu )

    
    # CPU - Informo el resultado.
    print("-------------------------------------------------")
    print("vector frecuencia: ")
    print(vector_frecuencias_cpu)
        
    mas_grande = 0
    valor_moda = 0

    # obtengo la moda del vector_frecuencias_cpu
    for rango in range(0, len(vector_frecuencias_cpu)):
        if(vector_frecuencias_cpu[rango] > mas_grande):
            mas_grande = vector_frecuencias_cpu[rango]
            valor_moda = rango
    
    print("-------------------------------------------------")
    print(f"Para esta ejecución la moda es: {valor_moda}, ")
    print(f"cual ocurrió {mas_grande} veces.")

    tiempo_total = datetime.now() - tiempo_total
    print("-------------------------------------------------")
    print("Tiempo Total: ", tiempo_en_ms(tiempo_total),"[ms]")
    print("Tiempo bucle: ", tiempo_en_ms(tiempo_bucle), "[ms]" )

except Exception as e:
  print(f"error: {e}")



-------------------------------------------------
vector respuestas cpu:
[1 3 7 ... 5 8 7]
Thread x:  256 , Bloque x: 1954
-------------------------------------------------
vector frecuencia: 
[46 46 47 47 47 47 47 47 50 50]
-------------------------------------------------
Para esta ejecución la moda es: 8, 
cual ocurrió 50 veces.
-------------------------------------------------
Tiempo Total:  14.647 [ms]
error: name 'tiempo_bucle' is not defined


#4. Tabla de pasos

Paso | Procesador | Funcion | Detalle
------------ | ------------ | ------------- | -------------
1 | CPU | @param | Lectura del tamaño de vector de Colab.
2 | CPU | import | Importa los módulos para funcionar.
3 | CPU | datetime.now() | Toma el tiempo actual.
4 | CPU | np.random.randint(0, 10, size = cantidad_elementos) | Inicializa el vector _vector_resultados_ con puntajes (de 0 a 9) aleatorios en cada elemento, en una cantidad de elementos ingresada en el paso 1.
5 | CPU | [0 for i in range(10)] | inicializo el vector frecuencia a 0, este vector representará en cada posición el valor del puntaje (entre 0 y 9) y su contenido representará la cantidad de apariciónes en el vector resultados. Por esta razon es necesario inicializarlo a cero, para luego sumar las frecuencias.
6 | CPU | print | muestro los valores del vector_resultados
7 | CPU | bucle for | contabilizo las frecuencias de puntajes en el vector_resultados.
8 | CPU | print | imprimo los resultados del vector frecuencia
9 | CPU | bucle for | obtengo el valor de la moda y la cantidad de repeticiones.
10 | CPU | print | muestro los resultados
11 | CPU | print | informo tiempos de ejecución.

#5. Conclusiones

TODO: Preparar y correr cuaderno en gpu y realizar comparación.....

#6. Bibliografía

[1] MARKDOWN SYNTAX Colab: PDF

[2] Introducción a Python: Página Colab

[3] Como Programar en C C++ y Java 4ta Edición Harvey M. Deitel & Paul J. Deitel 

[4] Biblioteca BLAS: Referencia

[5] Documentación PyCUDA: WEB

[6] Repositorio de PyCUDA: WEB